# Keyword Recommender

In [1]:
keywords = [
    "Climate",
    "Winter",
    "stadium",
]

### Use model to build vectors

In [2]:
from pymilvus import model

embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name='./all-MiniLM-L6-v2', # Specify the model name
    device='cpu' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
)

/home/sachin/projects/experiments/connectRecommender/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vectors = embedding_fn.encode_documents(keywords)

In [4]:
data = [
    {"id": i, "vector": vectors[i], "text": keywords[i], "person": f"XYZ{i}"}
    for i in range(len(vectors))
]

### Store into vector database

In [5]:
db_name = "keyword_store.db"
collection_name = "keyword_collection"
from pymilvus import MilvusClient
client = MilvusClient(db_name)

In [6]:
if client.has_collection(collection_name=collection_name):
    client.drop_collection(collection_name=collection_name)
client.create_collection(
    collection_name=collection_name,
    dimension=384,  # The vectors we will use in this demo has 768 dimensions
)

In [7]:
res = client.insert(collection_name=collection_name, data=data)

### Search from database based on Query

In [8]:
query_vectors = embedding_fn.encode_queries(["Connect me with a weather expert"])

res = client.search(
    collection_name=collection_name,  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "person"],  # specifies fields to be returned
)

In [9]:
res

data: ["[{'id': 0, 'distance': 0.5422773957252502, 'entity': {'text': 'Climate', 'person': 'XYZ0'}}, {'id': 1, 'distance': 0.3582354485988617, 'entity': {'text': 'Winter', 'person': 'XYZ1'}}]"] 

In [10]:
for d in res[0]:
    print(f"Keyword: {d['entity']['text']}")
    print(f"Person: {d['entity']['person']}")
    print(f"Score: {d['distance']}\n")

Keyword: Climate
Person: XYZ0
Score: 0.5422773957252502

Keyword: Winter
Person: XYZ1
Score: 0.3582354485988617

